<a href="https://colab.research.google.com/github/shivag/cs145-public/blob/main/Hashing_Algos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pybloom_live h3 faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.8/271.8 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.6 MB/s eta 0:00:00
  Created wheel for pybloom_live: filename=pybloom_live-4.0.0-py3-none-any.whl size=9233 sha256=40d30d5ca2e1f4b2edb252572cec0385eb9fd664954b4b50745061e734a3726c
  Stored in directory: /root/.cache/pip/wheels/fa/05/5b/632be6183ffb57a72bd83d7173079a0eb4cee8c010063199f8
Successfully built pybloom_live


In [7]:
import h3
# lat/lngs from SF mission, Stanford NVidia, Packard, Gates Buildings
locations = [(37.788022, -122.399797), (37.428226, -122.174722), 
             (37.429749, -122.1735490), (37.429761, -122.173290)]
for zoom in [8, 12]: # zoomlevels 8 and 12 (imagine the +/- in map zoom)
  for l in locations:
    cell_id = h3.geo_to_h3(l[0], l[1], zoom)
    print("@zoom[", zoom, "]:", cell_id)

@zoom[ 8 ]: 88283082a1fffff
@zoom[ 8 ]: 8828347417fffff
@zoom[ 8 ]: 8828347417fffff
@zoom[ 8 ]: 8828347417fffff
@zoom[ 12 ]: 8c283082a06bbff
@zoom[ 12 ]: 8c28347416257ff
@zoom[ 12 ]: 8c2834741602bff
@zoom[ 12 ]: 8c28347416021ff


In [8]:
from pybloom_live import BloomFilter

# Create a Bloom filter with 1000 items and a 0.1% false positive rate
bf = BloomFilter(capacity=1000, error_rate=0.001)

# Add some integers to the Bloom filter
bf.add(42); bf.add(30); bf.add(50)

# Check if each integer in the list is in the Bloom filter
integer_list = [42, 50, 100, 32]
for i in integer_list:
  if i in bf:
    print(f"{i} may be in the Bloom filter")
  else:
    print(f"{i} is definitely not in the Bloom filter")

42 may be in the Bloom filter
50 may be in the Bloom filter
100 is definitely not in the Bloom filter
32 is definitely not in the Bloom filter


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
import numpy as np, faiss
# Generate 1000 points in a 10-dimensional space
X = np.random.rand(1000, 10).astype('float32')

# Build an LSH index for the points
index = faiss.IndexLSH(10, 8)
index.add(X)

# Find the 5 nearest neighbors (distances and points) of xq, a new 10-dim point
xq = np.random.rand(10).astype('float32') 
D, I = index.search(np.expand_dims(xq, axis=0), k=5)

# Print the indices and distances of the nearest neighbors
print("Nearest neighbor of ", xq, " is:", I, " with distance: ", D)

Nearest neighbor of  [0.9562943  0.4509454  0.75692314 0.96686363 0.79955685 0.2184114
 0.5928665  0.6337663  0.41091037 0.99143934]  is: [[21 49 57 59 79]]  with distance:  [[0. 0. 0. 0. 0.]]


In [13]:
import faiss
import numpy as np
from PIL import Image

d = 256 # number of dimensions in the feature vector
# Map an image to a k-dimensions
def normed(fname):
  # Load the image
  img = Image.open(fname)
  img = img.resize((16, 16))
  img = np.array(img)
  # Convert the image to feature vector
  x = img.reshape(-1, d).astype('float32')
  faiss.normalize_L2(x) # Normalize the feature vector
  return x

# Build an index for the feature vectors
index = faiss.IndexFlatL2(d)
index.add(normed('stanford-logo.png'))
for img in ["stanford-logo.png", "stanford-logo2.jpg", "stanford-logo3.png"]:
  index.add(normed(img))
# Query an example image
query_x = normed('stanford-logo3.png')
D, I = index.search(query_x, k=2)
print(D)

[[0.         0.27528164]
 [0.         0.13537623]
 [0.         0.13537623]
 [0.         0.18201266]]
